# **Set according to environment (e.g. local, Google Colab...)**

In [1]:
project_folder = ''

# **Body**

In [2]:
from custom_libraries.miscellaneous import *
from custom_libraries.image_dataset import *
import numpy as np

In [3]:
from keras import backend
import math

def init_weights(masker):

    density = np.count_nonzero(masker == 1) / masker.size
    stddev = math.sqrt(2/(masker.shape[0]*density))
    weights = np.random.normal(size=masker.shape, loc = 0., scale=stddev)
    weights[np.where(masker == 0)] = 0
    weights = np.reshape(weights, [-1])
    weights = weights[np.where(weights != 0)]
    return weights

class treeLayer(tf.keras.layers.Layer):

    def __init__(self, Input_size=3072, use_bias=False):
        super(treeLayer, self).__init__()
        self.bias = None
        self.summer = None
        self.kernel = None
        self.Input_size = Input_size
        self.use_bias = use_bias

    def build(self, input_shape):

        self.summer = np.zeros((self.Input_size, self.Input_size // 2))
        for i in range(self.Input_size):
            self.summer[i, i // 2] = 1

        initializer = init_weights(self.summer)
        initializer = tf.keras.initializers.Constant(initializer)

        self.kernel = self.add_weight(shape=(1, self.Input_size),
                                      initializer=initializer,
                                      trainable=True, dtype=tf.float32)

        self.summer = tf.convert_to_tensor(self.summer, dtype=tf.float32)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.Input_size//2,),
                                        initializer=tf.keras.initializers.Zeros,
                                        trainable=True)

    def call(self, inputs):
        #print(inputs.shape, self.kernel.shape)
        x = tf.math.multiply(inputs, self.kernel)
        x = tf.matmul(x, self.summer)
        #masked_weights = tf.multiply(self.kernel, self.summer)
        #x = tf.matmul(inputs, masked_weights)
        if self.use_bias:
            x = tf.nn.bias_add(x, self.bias)

        x = tf.nn.leaky_relu(x, alpha = .01)
        return x

def create_model(input_size, num_trees=1, use_bias=False):
    model = tf.keras.Sequential()
    while input_size > num_trees:
        model.add(treeLayer(input_size, use_bias))
        input_size = input_size // 2
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', use_bias=use_bias))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-08),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.V2.AUTO),
                  metrics=['binary_crossentropy', 'acc'])

    return model


In [4]:
import gc

# Initialize settings
bs = 256
trials = 10
epochs = 2000
trees_set = [1, 32]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[3, 5, 'mnist'],
           [0, 6, 'fmnist'],
           [2, 6, 'kmnist'],
           [14, 17, 'emnist']]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
history = np.zeros((len(classes), trials, len(trees_set), 2))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test', data_dir=None)
    train_ds = ImageDataset(ds, 'train', data_dir=None)

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")
        X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                             validation_size=len(test_ds.images))
        X_test, y_test = test_ds.subset()
        test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(
            lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        #if history[j, -1, k, 0] != 0:
        #  continue

        for i in range(trials):
            print(f"Trial {i + 1}")

            #with tf.device('/device:GPU:0'):

            model = create_model(input_size=X_train[i].shape[1] * trees, num_trees=trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

            fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                    validation_data=valid_set, validation_batch_size=bs,
                                    callbacks=[callback], verbose=0)
            print_fit_history(fit_history, epochs)

            evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
            print_evaluate_history(evaluate_history)

            history[j, i, k] = evaluate_history[1:]

            np.save(project_folder + 'results/ktree_history.npy', history,
                    allow_pickle=True)

            del model, train_set, valid_set
            gc.collect()

Dataset: fmnist / Pair: 0-6
1-FCNN
Trial 1
Epochs: 262/2000 - Train BCE: 0.4025, accuracy: 82.9% - Validation BCE: 0.4825, accuracy: 79.0%
Test BCE: 0.4665, accuracy: 78.0%
Trial 2
Epochs: 61/2000 - Train BCE: 0.6931, accuracy: 50.7% - Validation BCE: 0.6938, accuracy: 48.0%
Test BCE: 0.6932, accuracy: 50.0%
Trial 3
Epochs: 357/2000 - Train BCE: 0.3944, accuracy: 83.7% - Validation BCE: 0.4608, accuracy: 79.5%
Test BCE: 0.4698, accuracy: 78.95%
Trial 4
Epochs: 61/2000 - Train BCE: 0.6924, accuracy: 51.8% - Validation BCE: 0.6961, accuracy: 46.0%
Test BCE: 0.6936, accuracy: 50.0%
Trial 5
Epochs: 521/2000 - Train BCE: 0.2595, accuracy: 92.2% - Validation BCE: 0.5041, accuracy: 83.5%
Test BCE: 0.4952, accuracy: 81.25%
Trial 6
Epochs: 433/2000 - Train BCE: 0.3634, accuracy: 85.4% - Validation BCE: 0.4434, accuracy: 80.5%
Test BCE: 0.4648, accuracy: 78.55%
Trial 7
Epochs: 484/2000 - Train BCE: 0.3157, accuracy: 87.9% - Validation BCE: 0.454, accuracy: 82.0%
Test BCE: 0.4502, accuracy: 79.35

KeyboardInterrupt: 

In [ ]:
history = np.load(project_folder + 'results/ktree_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-tree")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")